In [28]:
import pandas as pd
from yahoofinancials import YahooFinancials
import matplotlib.pyplot as plt
import requests
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from bs4 import BeautifulSoup

In [31]:
class DataRetrieval(object):
    
    def __init__(self,ticker):
        self.ticker = ticker.upper()
        self.financials = YahooFinancials(ticker)
        self.timeframe = 'annual'
        self.balance_sheet = None
        self.income_statement = None
        self.cash_flow_statement = None
        self.key_statistics = None
        self.historical_eps = None
        self.price_data = None
    
    def __set_balance_sheet(self):
        if self.balance_sheet == None:
            try:
                self.balance_sheet = self.financials.get_financial_stmts(self.timeframe,'balance')['balanceSheetHistory'][self.ticker]
                self.balance_sheet = self.__clean_statement_timestamp(self.balance_sheet)
            except:
                return "Error: Could not retrieve balance sheet"
    
    def __set_income_statement(self):
        if self.income_statement == None:
            try:
                self.income_statement = self.income_statement = self.financials.get_financial_stmts(self.timeframe,'income')['incomeStatementHistory'][self.ticker]
                self.income_statement = self.__clean_statement_timestamp(self.income_statement)
            except:
                return "Error: Could not retrieve income statement"
    
    def __set_cash_flow_statement(self):
        if self.cash_flow_statement == None:
            try:
                self.cash_flow_statement = self.cash_flow_statement = self.financials.get_financial_stmts(self.timeframe,'cash')['cashflowStatementHistory'][self.ticker]
                self.cash_flow_statement = self.__clean_statement_timestamp(self.cash_flow_statement)
            except:
                return "Error: Could not retrieve cash flow statement"
            
    def __set_key_statistics(self):
        if self.key_statistics == None:
            try:
                self.key_statistics = self.financials.get_key_statistics_data()
            except:
                return "Error: Could not retrieve key statistics"
    
    def __set_historical_eps(self):
        if self.historical_eps == None:
            url = 'https://www.nasdaq.com/earnings/report/{}'.format(self.ticker)
            page = requests.get(url)
            if page.status_code == 200:
                soup = BeautifulSoup(page.text,'html.parser')
                eps_list = []
                td = soup.find_all('td')
                counter = 0
                for i in td[2::5]:
                    try:
                        test = float(i.get_text())
                        counter += 1
                    except:
                        break
                for i in range(counter*5): 
                    eps_list.append(list(soup.find_all('td'))[i].get_text())
                self.historical_eps = pd.DataFrame({'textdate':eps_list[0::5],'timestamp':eps_list[1::5],'eps':eps_list[2::5],'consensus_eps':eps_list[3::5],'surprise':eps_list[4::5]})
                self.historical_eps = self.historical_eps.iloc[::-1]
                self.historical_eps.reset_index(inplace=True,drop=True)
            else:
                return "Error: Could not connect to NASDAQ website"
    
    def __set_price_data(self):
        if self.price_data == None:
            try:
                self.price_data = pd.DataFrame(self.financials.get_historical_price_data('1800-01-01',str(datetime.now())[:10],'daily')[self.ticker]['prices'])
            except:
                "Error: Could not retrieve historical price data"

    def __clean_statement_timestamp(self,statement):
        for i in statement:
                for j in i.keys():
                    i[j[:4]] = i.pop(j)
        return statement
    
    def __get_specific_year(self,statement,year):
        for i in statement:
            if str(list(i.keys())[0]) == str(year):
                return i['{}'.format(year)]
    
    def __json_to_dataframe(self,data):
        columns = []
        for i in data:
            for j in i.keys():
                columns.append(j)
        df = pd.DataFrame(data[0])
        for i in range(len(columns)):
            df[columns[i]] = pd.DataFrame(data[i])
        df = df[df.columns[::-1]]
        return df

    def get_balance_sheet(self):
        self.__set_balance_sheet()
        df = self.__json_to_dataframe(self.balance_sheet)
        return df
    
    def get_income_statement(self):
        self.__set_income_statement()
        df = self.__json_to_dataframe(self.income_statement)
        return df
    
    def get_cash_flow_statement(self):
        self.__set_cash_flow_statement()
        df = self.__json_to_dataframe(self.cash_flow_statement)
        return df
    
    def get_key_statistics(self):
        self.__set_key_statistics()
        ser = pd.Series(self.key_statistics[self.ticker])
        return ser
    
    def get_historical_eps(self):
        self.__set_historical_eps()
        return self.historical_eps
    
    def get_price_data(self):
        self.__set_price_data()
        return self.price_data
    
    def check_ticker(self):
        return self.ticker

    def revenue(self,year):
        self.__set_income_statement()
        try:
            return self.__get_specific_year(self.income_statement,year)['totalRevenue']
        except:
            return None

    def total_expenses(self,year):
        self.__set_income_statement()
        self.__set_cash_flow_statement()
        try:
            operating_expenses = self.__get_specific_year(self.income_statement,year)['totalOperatingExpenses']
            interest = self.__get_specific_year(self.income_statement,year)['interestExpense']
            tax = self.__get_specific_year(self.income_statement,year)['incomeTaxExpense']
            depreciation = self.__get_specific_year(self.cash_flow_statement,year)['depreciation']
            return Formulas.total_expenses(operating_expenses,interest,tax,depreciation)
        except:
            return None
            
    
    def operating_expenses(self,year):
        self.__set_cash_flow_statement()
        try:
            return self.__get_specific_year(self.cash_flow_statement,year)['totalOperatingExpenses']
        except:
            return None
    
    def outstanding_shares(self):
        self.__set_key_statistics()
        try:
            return self.key_statistics[self.ticker]['sharesOutstanding']
        except:
            return None
    
    def share_price(self):
        pass
    
    def market_value(self):
        pass
    
    def liquid_assets(self):
        pass
    
    def total_debt(self):
        pass
    
    def total_debt(self):
        pass
    
    def tax(self):
        return Formulas.gross_profit(self.revenue(),self.operating_expenses()) * (1-self.corporate_tax_rate)
    
    def interest(self):
        pass
    
    def depreciation(self):
        pass
    
    def amortization(self):
        pass
    
    def cost_of_goods_sold(self):
        pass
    
    def receivables(self):
        pass
    
    def liabilities(self):
        pass
    
    def total_assets(self):
        pass
    
    def total_capital(self):
        pass
    
    def total_equity(self):
        pass
    
    def capital_expenditures(self):
        pass


In [35]:
f = figureRetrieval('cvx')

In [397]:
f.revenue(2018)

265595000000

In [370]:
f.check_ticker()

'CVX'

In [ ]:
class RatioCalculation(object):
    
    def __init__(self):
        self.corporate_tax_rate = 0.21
        self.risk_free_ror = 0.025
        self.market_ror = 0.098
    
    def set_corporate_tax_rate(self,corporate_tax_rate):
        self.corporate_tax_rate = corporate_tax_rate
        
    def set_risk_free_ror(self,risk_free_ror):
        self.risk_free_ror = risk_free_ror
        
    def set_market_ror(self,market_ror):
        self.market_ror = market_ror

In [ ]:
class TrendModeling(object):
    # this class should model line of best fit trends on first level,
    # and second level acceleration line of best fit trends

For modelling, use a system wherby you can freely adjust the growth parameters, and if you so choose - can use historical growth/acceleration metrics to adjust those parameters for you automatically

chapter 12 should be for modelling 

In [ ]:
class ValueModeling(object):
    

In [ ]:
class RiskModeling(object):
    

In [412]:
# Visualization class
class Visualize:
    #df = df.T.iloc[::-1]
    pass

In [ ]:
class Recommend(object):
    
    def __init__(self,company):
        company.runall()

In [38]:
class Company(object):
    def __init__(self,ticker):
        self.ticker = ticker
        self.financials = YahooFinancials(self.ticker)
        
    # Gather all possible quantitative metrics available    
    def runall():
        